In [2]:
import pandas as pd
import numpy as np

In [3]:
aluminum_division = 12
near_k = 2

In [4]:
df = pd.read_csv(r"C:\Users\tzuli\Documents\python\teco\data\train\train_data.csv")
print(df.shape)
df.head(3)

(179, 16)


,日期,工號,序號,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,平衡_F側角度,平衡_F側配重,最終_L側角度,最終_L側不平衡量,最終_F側角度,最終_F側不平衡量
0,2023-06-07,FC236001,1,1000,277.0,12.5,145.0,24.9,277.0,12.0,145.0,24.0,312,1.4,56,2.2
1,2023-06-07,FC236001,2,1000,312.0,1.4,56.0,2.2,312.0,1.0,146.0,2.0,304,1.8,98,1.1
2,2023-06-07,FC234613,1,1000,298.0,155.0,267.0,64.3,298.0,150.0,267.0,60.0,9,23.0,197,3.6


In [5]:
df_total = df.groupby("工號").first()
df_total1 = df.groupby("工號").agg({"平衡_L側配重": sum, "平衡_F側配重": sum})
df_total1.columns = ["平衡_L側加總", "平衡_F側加總"]
df_total = pd.merge(df_total, df_total1, left_index = True, right_index = True)
df_total = df_total.reset_index().iloc[:, [0, 5, -2, 9, 7, -1, 11]]
df_total.head(3)

,工號,初始_L側不平衡量,平衡_L側加總,平衡_L側配重,初始_F側不平衡量,平衡_F側加總,平衡_F側配重
0,FC225607-1,114.0,136.0,120.0,122.0,137.0,120.0
1,FC225607-2,317.0,416.0,300.0,113.0,122.0,100.0
2,FC225607-3,130.0,154.0,120.0,174.0,202.0,160.0


In [6]:
angle_init = np.linspace(0, 360, (aluminum_division + 1)).astype(int)
angle_col = [f"L_{i}" for i in angle_init[:-1]] + [f"F_{i}" for i in angle_init[:-1]]
df[angle_col] = None
df.head(3)

,日期,工號,序號,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,...,F_60,F_90,F_120,F_150,F_180,F_210,F_240,F_270,F_300,F_330
0,2023-06-07,FC236001,1,1000,277.0,12.5,145.0,24.9,277.0,12.0,...,None,None,None,None,None,None,None,None,None,None
1,2023-06-07,FC236001,2,1000,312.0,1.4,56.0,2.2,312.0,1.0,...,None,None,None,None,None,None,None,None,None,None
2,2023-06-07,FC234613,1,1000,298.0,155.0,267.0,64.3,298.0,150.0,...,None,None,None,None,None,None,None,None,None,None


In [7]:
for i in range(len(df)):
    angle = df.loc[i, "初始_L側角度"]
    weight = df.loc[i, "平衡_L側配重"]
    closest_index = np.argsort(np.abs(angle_init - angle))[:near_k] # 與angle_ori最接近的K個角度
    near_angle = angle_init[closest_index]
    gap = (angle - near_angle[0]) / (near_angle[1] - near_angle[0])

    if near_angle[0] == 360:
        near_angle[0] = 0
    if near_angle[1] == 360:
        near_angle[1] = 0

    if gap < 0.3:
        df.loc[i, f"L_{near_angle[0]}"] = weight
    elif gap > 0.7:        
        df.loc[i, f"L_{near_angle[1]}"] = weight
    else:
        df.loc[i, f"L_{near_angle[0]}"] = weight/2
        df.loc[i, f"L_{near_angle[1]}"] = weight/2

In [8]:
for i in range(len(df)):
    angle = df.loc[i, "初始_F側角度"]
    weight = df.loc[i, "平衡_F側配重"]
    closest_index = np.argsort(np.abs(angle_init - angle))[:near_k] # 與angle_ori最接近的K個角度
    near_angle = angle_init[closest_index]
    gap = (angle - near_angle[0]) / (near_angle[1] - near_angle[0])

    if near_angle[0] == 360:
        near_angle[0] = 0
    if near_angle[1] == 360:
        near_angle[1] = 0

    if gap < 0.3:
        df.loc[i, f"F_{near_angle[0]}"] = weight
    elif gap > 0.7:        
        df.loc[i, f"F_{near_angle[1]}"] = weight
    else:
        df.loc[i, f"F_{near_angle[0]}"] = weight/2
        df.loc[i, f"F_{near_angle[1]}"] = weight/2

In [9]:
for i in range(len(df)):
    if df.loc[i, "序號"] != 1:
        df.loc[i, "工號"] = None
        df.loc[i, "日期"] = None
df.head()

,日期,工號,序號,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,...,F_60,F_90,F_120,F_150,F_180,F_210,F_240,F_270,F_300,F_330
0,2023-06-07,FC236001,1,1000,277.0,12.5,145.0,24.9,277.0,12.0,...,None,None,None,24.0,None,None,None,None,None,None
1,None,None,2,1000,312.0,1.4,56.0,2.2,312.0,1.0,...,2.0,None,None,None,None,None,None,None,None,None
2,2023-06-07,FC234613,1,1000,298.0,155.0,267.0,64.3,298.0,150.0,...,None,None,None,None,None,None,None,60.0,None,None
3,None,None,2,1000,9.0,23.0,197.0,3.6,9.0,20.0,...,None,None,None,None,1.5,1.5,None,None,None,None
4,None,None,3,1000,106.0,3.7,178.0,2.7,275.0,-3.0,...,None,None,None,None,3.0,None,None,None,None,None


In [10]:
writer = pd.ExcelWriter(r"C:\Users\tzuli\Documents\python\teco\data\train\eda.xlsx", engine = 'xlsxwriter')
df.to_excel(writer, sheet_name = '各角度填補', index = False)
df_total.to_excel(writer, sheet_name = '填補加總', index = False)
writer.close()